
<h3><p style="text-align: center;">JANUS - Joint ANalysis for augmentation of clUSter specificity</p></h3>
<p style="text-align: center;">- DepMap and subset of DepMap samples -</p>
<p style="text-align: center;">Updated: 26/05/2023</p>



Package dependencies 
===

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import norm, pearsonr
import helpers as h
from tqdm.auto import tqdm
tqdm.pandas()
import os
import slugify
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import kaleido


Data preparation and preprocessing   
===
X12 = Achilles_gene_effect_20Q2 / ~700 samples    
X1 = DepMap - Cancer Samples  
X2 = Subset of DepMap for cancer specific samples / 27 subset

We Apply z-score normalization for X12, since other X1 and X2 is the subset of this dataset, we do not need to apply separately.  
We delete paranteses from DepMap dataset.  
There are some empty cells, we drop this cells sample wise. It does not effect gene size, reduceses sample size a bit.  



In [ ]:
df = pd.read_csv('./input/Achilles_gene_effect_20Q2.csv', index_col=0).T
# delete paranteses from gene names
df.index = [i.split(' ')[0] for i in df.index] # type: ignore
# zscore norm. for only X1
dfnorm = (df - df.mean())/df.std(ddof=0)
df = dfnorm.T.dropna().T

Read go terms from database
===

- Read terms CSV files, create set of genes exists in the terms. 
- Check how many unique genes exist in the terms
- Use only common genes (between terms and datasets) for further analysis to avoid zero hits in the PR analysis
- Exclude terms with 2 genes or below

In [ ]:
terms, terms_uniq_genes = h.read_terms("./input/terms/data_complex.csv")
# common genes with DepMap and TermsDB
common_genes = list(set(terms_uniq_genes) & set(df.index))
# filter complexes
#terms = terms.query('Length > 2').reset_index(drop=True)

DepMap sample annotation
===

We use DepMap annotation file and annotate each sample and group them disease wise.  
We do not include 1 sampled disease.

In [ ]:
diseases = pd.read_excel('./input/types.xlsx')
diseases = diseases[diseases.include ==1 ]
info = pd.read_csv('./input/Achilles_gene_effect_20Q2_info.csv',index_col=0)
result_dir = 'C:/Users/yd/Desktop/projects/janus/results/per_disease/'
common_dir = os.path.join(result_dir,'common')

DO annotation for X12 once, copy this annotation to all diseases. 
----

__This area only changes when TERMS are changed !__  
We do annotation for X12 (DepMap) once, hence we will check absence of gene pairs in the Terms database.  
Since our gene count does not change (it only changes when terms changed, because we use common genes with terms), gene pairs also will not change. So, we will use this annotation to annotate X1 and X2.

+ perform correlation and save it in the common folder  
+ filter using common_genes  
+ convert to the Triangular matrix (this is important for the pairwise section because we want to exclude self and mirror correlations)  
+ make pairwise  
+ annotation (we use parallel function to make it faster)  
+ run PR analysis and save it  

In [ ]:
# this area only changes when TERMS changed
#x12 = h.perform_corr(df)
#x12.to_parquet(os.path.join(common_dir,'x12.p'))
#f12 = x12.loc[common_genes,common_genes].copy()
#h.convert_full_to_half_matrix(f12)
#s12 = h.make_binary(f12)
#annotated12 = h.run_annotation_parallel(s12,terms, 10, 10)
#pra12 = h.pr_analysis(annotated12)
#pra12.to_parquet(os.path.join(common_dir,'pra12.p')) 
# this area only changes when TERMS changed

# this area only changes when TERMS changed.
if os.path.isfile(os.path.join(common_dir,'x12.p')):
     x12 = pd.read_parquet(os.path.join(common_dir,'x12.p'))
     pra12 = pd.read_parquet(os.path.join(common_dir,'pra12.p'))
     annotated12 = pra12.drop(['prediction','tp','precision','recall'],axis=1)
     f12 = x12.loc[common_genes,common_genes].copy()
     h.convert_full_to_half_matrix(f12)
     s12 = h.make_binary(f12)


Correlation and PR analysis [each disease]
---

For each disease (n_disease:27), separate samples and perform correlation analysis  
Perform PR analysis and save the files  

In [ ]:
for k,row in tqdm(diseases.iterrows()):  
    x1, x2 = h.get_disease_corr_matrix(df, row.disease)
    pra1 = h.pr_analysis_quick(x1, common_genes, annotated12, row.disease, 'pra1.p')
    pra2 = h.pr_analysis_quick(x2, common_genes, annotated12, row.disease, 'pra2.p')

Per-Complex PR Analysis [for DepMap]
---

Perform this analysis for each row (complex/pathway/GOBP) 
- check how many genes exist in genome (because some complex genes may not be in the dataset)
- contunie analysis if checked gene count is above 2 (if it is not, assign set NAN)
- Filter correlation matrix [complex_genes, genome_genes]
- convert to pairwise
- drop mirror pairs A,B and B,A (otherwise we will get double TP count)
- sort pcc score of pairs

Calculation of TP, Precision, Recall

- TP: Check sorted values in order, If it is anottated in the Terms database it is TP
- Precision: TP / Index
- Recall: TP / Number of Total TP

After finding of Precision & Recall, calculate AUC score.   
https://scikit-learn.org/stable/modules/generated/sklearn.metrics.auc.html

In [ ]:
auc12 = terms.set.progress_apply(lambda g: h.pra_single_complex(x12, common_genes, g, 3))
score, n_gene_used = list(zip(*auc12))
terms['n_gene_used'] = n_gene_used
terms['auc12'] = score
terms = terms.drop(['ID','list','set'],axis=1)
#terms.to_parquet(os.path.join(result_dir, common_dir, 'per_complex_12.p'))

Per-Complex PR Analysis [each disease]
---
Here we perform PerComplex PR analysis and Calculate AUC.  
Since DepMap dataset is common for each disease, we only calculate for X1 and X2.

In [ ]:
for k,row in tqdm(diseases.iterrows()):  
    terms, terms_uniq_genes = h.read_terms("./input/terms/data_complex.csv")

    x1, x2 = h.get_disease_corr_matrix(df, row.disease)
    auc1 = terms.set.progress_apply(lambda g: h.pra_single_complex(x1, common_genes, g, 3))
    auc2 = terms.set.progress_apply(lambda g: h.pra_single_complex(x2, common_genes, g, 3))
    
    score1, n_gene_used = list(zip(*auc1))
    score2, n_gene_used = list(zip(*auc2))
    terms['auc1'] = score1
    terms['auc2'] = score2
    terms = terms.drop(['ID','list','set'],axis=1)
    terms.to_parquet(os.path.join(result_dir, slugify.slugify(row.disease), 'per_complex_1and2.p'))


Perform Correlation Contribution Formula [each disease]
---

- Use custom correlation function to check contributions of datasets
- save denominator and contribution integers beside the contribution score

In [ ]:
for k,row in tqdm(diseases.iterrows()):  
    print(f'{row.disease} | contribution analysis is started..' )
    save_dir = os.path.join(result_dir, slugify.slugify(row.disease), 'contr')
    os.makedirs(save_dir,exist_ok=True)
    d1, d2 = h.get_separated_datasets(df, row.disease)
    n_small_set = d2.shape[1]
    dfmer = pd.merge(d2,d1,left_index=True,right_index=True)
    contr_big, contr_small, _, _, denom = h.partial_corr(dfmer, n_small_set)
    contr_big.to_parquet(os.path.join(save_dir, 'contr_big.p'))
    contr_small.to_parquet(os.path.join(save_dir, 'contr_small.p'))
    denom.to_parquet(os.path.join(save_dir, 'denom.p'))  
        

PCC mean of Complex Genes
---

For each row (complex/pathway/GOBP) calculate mean of PCC values of given genes   
(i.e. if complex has 5 genes, get those 5 genes correlation from big correlation matrix which is 10 gene pairs. ((5*5) - 5) / 2)

- remove self correlations
- remove mirror pairs correlations

In [ ]:
selected_disease = ['Sarcoma','Pancreatic Cancer','Neuroblastoma']

x12 = pd.read_parquet(f'./results/per_disease/common/x12.p')
x12 = h.convert_full_to_half_matrix(x12)
per_complex_12 = pd.read_parquet(f'./results/per_disease/common/per_complex_12.p')

for k,row in tqdm(diseases.iterrows()):  

    # if row.disease not in selected_disease:
    #     continue

    print(row.disease)
    terms, terms_uniq_genes = h.read_terms("./input/terms/data_complex.csv")

    x1 = pd.read_parquet(f'./results/per_disease/{row.slug}/x1.p')
    x2 = pd.read_parquet(f'./results/per_disease/{row.slug}/x2.p')
    contr_small = pd.read_parquet(f'./results/per_disease/{row.slug}/contr/contr_small.p')
    contr_big = pd.read_parquet(f'./results/per_disease/{row.slug}/contr/contr_big.p')
    x1 = h.convert_full_to_half_matrix(x1)
    x2 = h.convert_full_to_half_matrix(x2)
    per_complex_1and2 = pd.read_parquet(f'./results/per_disease/{row.slug}/per_complex_1and2.p')

    data = {
      'x12': x12,
      'x1': x1,
      'x2': x2,
      'contr_small': contr_small,
      'contr_big': contr_big,        
    }

    for data_var,v in tqdm(data.items()):    
        df = v.copy()
        for k,r in tqdm(terms.iterrows()):
            terms.at[k,data_var] = h.get_mean_selected_genes(r.set, df, 3)

    terms['pr_auc_x1'] = per_complex_1and2.auc1
    terms['pr_auc_x2'] = per_complex_1and2.auc2
    terms['pr_auc_x12'] = per_complex_12.auc12

    terms.to_parquet(f'./results/per_disease/{row.slug}/mean.p')
    terms.drop(['list','set'],axis=1).to_csv(f'./results/per_disease/{row.slug}/mean.csv',na_rep='NA')


Scatter plot of AUC1 vs AUC2 [each disease]
---

In [ ]:
selected_disease = ['Sarcoma','Pancreatic Cancer','Neuroblastoma']
selected_disease = ['Myeloma']

for k,row in tqdm(diseases.iterrows()): 

    # if row.disease not in selected_disease:
    #     continue   
    
    terms = pd.read_parquet(f'./results/per_disease/{row.slug}/mean.p')
    terms = terms[~terms.x1.isna()]
    terms = terms.drop(['list','set'],axis=1)
    terms['hue'] = 0
    terms = terms.sort_values('contr_small', ascending=False)
    terms.iloc[:10, terms.columns.get_loc('hue')] = 2 
    terms = terms.sort_values('contr_big', ascending=False)
    terms.iloc[:10, terms.columns.get_loc('hue')] = 1 
    terms['hue2'] = terms.hue.map({0: 'normal', 1:'big_contr_extreme_10', 2:'small_contr_extreme_10'})
    # define legend for extreme values
    terms['legend'] = np.where(terms.hue != 0, terms['Name'].apply(lambda x: x[:20]), '')
 

    config = {'staticPlot': True}
    fig = go.Figure()

    fig = px.scatter(terms, x='pr_auc_x1', y='pr_auc_x2', color='hue2', 
    title=f'{row.disease} | n_sample: {row.n_sample}', color_discrete_map={
                 "normal": "grey","small_contr_extreme_10": "blue","big_contr_extreme_10": "black",
    })
    fig.add_shape( type="line", x0=0, y0=0, x1=1,y1=1, line=dict(color="Grey",width=1) )

    fig.update_layout(
        autosize=False,width=900, height=600,
        title_x=0.5,
        font_color="black",
        title_font_color="black",
    )

    fig.update_layout(
        xaxis=dict(domain = [0, 0.65],  anchor = 'y1'),
        legend=dict(
            title=" Contribution score",
            yanchor="top",
            y=0.99,
            xanchor="right",
            x=0.92),
            annotations=[
                go.layout.Annotation(
                    text="..<br>".join([i for i in terms[terms.hue == 1].legend]),
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.92,
                    y=0.58,
                    bordercolor='black',
                    borderwidth=1,
                    width=180,
                ),
                go.layout.Annotation(
                    text="..<br>".join([i for i in terms[terms.hue == 2].legend]),
                    align='left',
                    showarrow=False,
                    xref='paper',
                    yref='paper',
                    x=0.92,
                    y=0.0,
                    bordercolor='blue',
                    borderwidth=1,
                    width=180,
                )
            ],

    )

    fig.show(config=config)
    fig.write_image(f'{row.slug}.png',scale=3)
